# Build 3 Layer Network

- input layer
- 2 hidden layer
- output layer
<br>
<br>
- Loss function : Cross Entropy Error

In [1]:
import numpy as np

## Define Functions

In [2]:
# sigmoid

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [3]:
a = np.array([[1, 2, 5, 6, 3],
             [3, 5, 62, 1, 2],
             [2, 3, 4, 5, 10]])
a.argmax(axis = 1)

array([3, 2, 4])

In [4]:
# cross_entropy_error
# y : predicted value
# t : label

def cross_entropy_error(y, t):
    
    # y: 신경망 출력
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    # t: 숫자 레이블인 경우
    if t.size == y.size:
        t = t.argmax(axis=1)
        
    delta = 1e-7
    
    return  -np.sum(np.log(y[np.arange(batch_size), t] + delta)) / batch_size

In [5]:
a = np.array([[1, 2, 5, 6, 3],
             [3, 5, 62, 1, 2],
             [2, 3, 4, 5, 10]])

In [6]:
nt = np.max(a, axis = 1)

In [7]:
t = np.max(a.T, axis = 0)

In [8]:
a.T - nt

array([[ -5, -59,  -8],
       [ -4, -57,  -7],
       [ -1,   0,  -6],
       [  0, -61,  -5],
       [ -3, -60,   0]])

In [16]:
# softmax
# x : input
# Overflow 방지 -> input 중 가장 큰 값 전체 데이터에서 빼주기

def softmax(x):
    
    if x.ndim == 2:
        #Broadcasting을 하기 위해
        x = x.T - np.max(x, axis = 1)
        y = np.exp(x) / np.sum(np.exp(x), axis = 0)
        return y.T
        
    c = np.max(x)
    x = x - c    
    return  np.exp(x) / np.sum(np.exp(x))

In [10]:
x = [[1, 2, 3],
    [3, 4, 5]]

x = np.array(x)

for idx in range(x.shape[0]):
    print(x[idx])
    print(x[idx].shape)

[1 2 3]
(3,)
[3 4 5]
(3,)


In [11]:
# numerical_gradient
# 수치 미분법
# f : function
# x : variable
# 각 변수에 대한 편미분 저장된 배열 return

def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    
    
    iter = np.nditer(x, flags = ['multi_index'], op_flags = ['readwrite'])
    with iter:
        while not iter.finished:
            idx = iter.multi_index
            tmp_val = x[idx]
            x[idx] = float(tmp_val) + h
            fxh1 = f(x)
            
            x[idx] = float(tmp_val) - h
            fxh2 = f(x)
            
            grad[idx] = (fxh1 - fxh2) / (2 * h)
            
            x[idx] = tmp_val
        return grad

## Define Network by Class

In [14]:
class ThreeLayerNet:
    # 초기화
    def __init__(self, input_size, hidden_size_1, hidden_size_2, output_size, weight_init_std=0.01):
        
        # initialize parameters
        # W -> random 
        # b -> 0
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size_1)
        self.params['b1'] = np.zeros(hidden_size_1)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size_1, hidden_size_2)
        self.params['b2'] = np.zeros(hidden_size_2)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size_2, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        
    # 예측 함수
    # x : 입력 데이터
    # y : 예측값
    def predict(self,x):
        
        #### 채워 넣으세요 ####
        # activation function : output 이전은 sigmoid
        W1, W2, W3 = self.params['W1'], self.params['W2'], self.params['W3']
        b1, b2, b3 = self.params['b1'], self.params['b2'], self.params['b3']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        z2 = sigmoid(a2)
        a3 = np.dot(z2, W3) + b3
        y = softmax(a3)
        
        return y
    
    
    # 손실 함수
    # cross_entropy_error 사용
    # x : 입력 데이터
    # t : 정답 레이블
    def loss(self, x, t):
        
        #### 채워 넣으세요 ####
        y = self.predict(x)
        
        return  cross_entropy_error(y, t)
    
    
    # 정확도 함수
    # x : 입력 데이터
    # t : 정답 레이블
    # x 중 예측값 == 정답 레이블 비율
    # accuracy = (정답 맞춘 데이터 수) / (전체 데이터 수)
    def accuracy(self, x, t):
        
        #### 채워 넣으세요 ####
        y = self.predict(x)
        y = np.argmax(y, axis = 1)
        t = np.argmax(t, axis = 1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return  accuracy
    
    
    # 기울기 함수
    # 수치 미분법으로 계산
    # x : 입력 데이터
    # t : 정답 레이블
    def gradient(self, x, t):
        # get loss function
        loss_W = lambda W: self.loss(x,t)
        
        grads = {}
        grads['W1'] =  numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] =  numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] =  numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] =  numerical_gradient(loss_W, self.params['b2'])
        grads['W3'] =  numerical_gradient(loss_W, self.params['W3'])
        grads['b3'] =  numerical_gradient(loss_W, self.params['b3'])
        
        #### 채워 넣으세요 ####
        
        return grads

## train, test - MNIST

교재 github에서 /dataset/mnist.py 다운로드 후 load_mnist import해서 사용

**ThreeLayerNet으로 train, test**

- input_size = 784
- hidden_size_1 = 50
- hidden_size_2 = 100
- output_size = 10
- batch_size = 100
- learning_rate = 0.1
- iters_num = 5000
- train_size = x_train data 전체

**train loss 그래프 그리기**

In [17]:
import sys, os
systrain_acc_listppend(os.pardir)

from mnist import load_mnist

# load dataset
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []
train_acc_list = []
test_acc_list = []

# Hyperparameters
#### 채워 넣으세요 ####
iters_num = 5000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

network = ThreeLayerNet(784, 50, 100, 10)

# 1 epoch 되는 iteration
iter_per_epoch = max(train_size/batch_size, 1)

# Train
for i in range(iters_num):
    # get minibatch
    # not random
    x_batch = x_train[i%train_size:i%train_size+batch_size]
    t_batch = t_train[i%train_size:i%train_size+batch_size]
    
    #### 채워 넣으세요 ####
    # gradient 계산 후 parameter update 필요
    grad = network.gradient(x_batch, t_batch)
    
    for i in ('W1', 'b1', 'W2', 'b2', 'W3', 'b1'):
        networkd.params[i] -= learning_rate * grad[i]
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    
    # 1 epoch 당 accuracy 계산
    if i%iter_per_epoch == 0:
        #### 채워 넣으세요 ####
        # train accuracy, test accuracy 각각 계산 후 (모델에서 정의한 accuracy 함수 이용)
        # train_acc_list, test_acc_list에 각각 accuracy append
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, y_test)
        
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        
        
        print("Train Accuracy: " + str(train_acc) + " & " + "Test Accuracy: " + str(test_acc))

KeyboardInterrupt: 

### Visualize Loss
- loss 보여주는 plot

- 맘대로 시각화하면 됩니다

In [ ]:
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label = 'trainacc')
plt.plot(x, test_acc_list, label = 'test acc', linestyle = '--')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(0, 1.0)
plt.legend(loc = 'lower right')
plt.show()